### Notebook for plots and figures

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import seaborn as sns

sns.set_theme("paper")


from cats.evaluation import *

MASTER_SEED = 235790

In [ ]:
# Generate Random Seeds


def generate_random_seeds(n: int):
    rng = np.random.default_rng(MASTER_SEED)
    return rng.integers(0, 2**32 - 1, size=(n,))


seeds = generate_random_seeds(5)

print(seeds)

In [ ]:
# fig:kitten-intrinsic-mcc

WINDOW = 5

rewards_hopper = pd.read_csv("data/kitten-im-mcc/evaluation_reward.csv").T

# Data Cleanup
im = ["baseline", "rnd", "disagreement", "icm"]
im_reg = "|".join(im)
rewards_hopper = rewards_hopper.filter(regex="^(?!.*(MIN|MAX|Step)).+", axis=0)
rewards_hopper = rewards_hopper.reset_index()
rewards_hopper["index"] = rewards_hopper["index"].str.extract(f"^.+({im_reg}).+$")
rewards_hopper_mean = rewards_hopper.groupby("index").mean().T
rewards_hopper_std = rewards_hopper.groupby("index").std().T

# Plot
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(10, 4)
fig.subplots_adjust(wspace=0.3)

# MCC
ax: Axes = axs[0]
colours = sns.color_palette()
for i, col in enumerate(im):
    c = colours[i]
    mean_ma = rewards_hopper_mean[col].rolling(window=WINDOW).mean()
    std_ma = rewards_hopper_std[col].rolling(window=WINDOW).mean()

    x = list(range(len(mean_ma)))
    ax.plot(x, mean_ma, label=col, color=c)
    ax.fill_between(x, mean_ma - std_ma, mean_ma + std_ma, alpha=0.2)

ax.set_title("Mountain Car Continuous")
ax.legend()
ax.set_xlabel("Step")
ax.set_ylabel("Extrinsic Reward per Episode (Rolling Mean 5)")

# ICM
rewards_mcc_icm = rewards_hopper[rewards_hopper["index"] == "icm"]
rewards_mcc_icm = rewards_mcc_icm.T.drop("index")
rewards_mcc_icm = rewards_mcc_icm.rolling(window=WINDOW).mean()

ax: Axes = axs[1]
ax.plot(rewards_hopper_mean["icm"].rolling(window=WINDOW).mean(), color=colours[3])
ax.plot(rewards_mcc_icm, "--", alpha=0.5)

ax.set_title("Mountain Car Continuous (ICM)")
ax.set_xlabel("Step")
ax.set_ylabel("Extrinsic Reward per Episode (Rolling Mean 5)")

fig.savefig("mcc_kitten_intrinsic.pdf", format="pdf", bbox_inches="tight")

In [ ]:
# fig: kitten-intrinsic-hopper

WINDOW = 20

rewards_hopper = pd.read_csv("data/kitten-im-hopper/evaluation_reward.csv").T

# Data Cleanup
im = ["baseline", "rnd", "disagreement", "icm"]
im_reg = "|".join(im)
rewards_hopper = rewards_hopper.filter(regex="^(?!.*(MIN|MAX|Step)).+", axis=0)
rewards_hopper = rewards_hopper.reset_index()
rewards_hopper["index"] = rewards_hopper["index"].str.extract(f"^.+({im_reg}).+$")
rewards_hopper_mean = rewards_hopper.groupby("index").mean().T
rewards_hopper_std = rewards_hopper.groupby("index").std().T

# Plot
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(8, 4)
fig.subplots_adjust(wspace=0.3)

# MCC
colours = sns.color_palette()
for i, col in enumerate(im):
    c = colours[i]
    mean_ma = rewards_hopper_mean[col].rolling(window=WINDOW).mean()
    std_ma = rewards_hopper_std[col].rolling(window=WINDOW).mean()

    x = list(range(len(mean_ma)))
    ax.plot(x, mean_ma, label=col, color=c)
    ax.fill_between(x, mean_ma - std_ma, mean_ma + std_ma, alpha=0.2)

ax.set_title("Hopper")
ax.legend()
ax.set_xlabel("Step")
ax.set_ylabel("Extrinsic Reward per Episode (Rolling Mean 20)")

fig.savefig("mcc_kitten_intrinsic_hopper.pdf", format="pdf", bbox_inches="tight")

In [ ]:
# ddpg-mcc-failure

from hydra import initialize, compose
from kitten.rl.qt_opt import cross_entropy_method

from cats.run import run

with initialize(version_base=None, config_path="cats/config"):
    cfg = compose(
        config_name="defaults_off_policy.yaml",
        overrides=[
            "seed=0",
            "env.max_episode_steps=999",
            "algorithm.type=ddpg",
            "cats.enable_policy_sampling=false",
            "intrinsic=disagreement",
            "cats.death_not_end=false",
            "train.total_frames=1024",
            "noise.scale=0.1",
        ],
    )
experiment = run(cfg)

fig, axs = plt.subplots(1, 4)
fig.set_size_inches(14, 2)
fig.subplots_adjust(wspace=0.7)

visualise_experiment_value_estimate(experiment, fig, axs[0])
visualise_memory(experiment, fig, axs[1])
visualise_experiment_policy(experiment, fig, axs[2])

# Oracle on learnt Critic
ax = axs[3]
s, states, x_label, y_label = generate_2d_grid(experiment)
# cross
actions = cross_entropy_method(
    s_0=s,
    critic_network=experiment.algorithm.critic,
    action_space=experiment.env.action_space,
    n=64,
    m=6,
    n_iterations=2,
    device=experiment.device,
)[:, :1]
norm = mpl.colors.Normalize(vmin=-1, vmax=1)
cmap = cm.viridis
m = cm.ScalarMappable(norm=norm, cmap=cmap)
colors = m.to_rgba(actions.detach().cpu())
ax.scatter(states[:, 0], states[:, 1], c=colors)
ax.set_title("Oracle Actions")
ax.set_xlabel(x_label)
ax.set_ylabel(y_label)
cbar = fig.colorbar(m, ax=ax)
cbar.set_label("Action", rotation=270, labelpad=15)

fig.savefig("ddpg_mcc_failure.pdf", format="pdf", bbox_inches="tight")

In [ ]:
# table:cats-derailment

from cats.on_policy_experiment import OnlineExperiment